# E.D.A Dataset train

En este notebook hacemos el E.D.A al dataset original train.csv, esto con el objetivo de Comprender los datos, Evaluar la calidad de los datos, Limpiar los datos, Reducir su complejidad y Sacar conclusiones sobre el contenido de este

#### Añadimos el directorio raíz al sys.path para facilitar importaciones

In [1]:
import os

notebook_dir = os.getcwd()
root_dir = os.path.abspath(os.path.join(notebook_dir, '..'))
os.chdir(root_dir)

print("Directorio de trabajo actual después del cambio:", os.getcwd())

Directorio de trabajo actual después del cambio: c:\Users\mirlo\proyecto1


#### Importamos las Librerias

In [20]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

#### Importamos la conexion 

In [5]:
from Database.conexion_BD import create_connection

connection = create_connection()
if connection is not None:
    print("Conexión establecida con éxito.")
else:
    print("No se pudo establecer la conexión.")

Conexión exitosa a la base de datos PostgreSQL.
Conexión establecida con éxito.


#### Cargamos la tabla flights ubicada en la Base de Datos PostgreSQL, y la almacenamos en un Dataframe 

In [6]:
table_name = 'flights' 

query = f"SELECT * FROM {table_name};"
flights = pd.read_sql(query, con=connection)
flights.head()

C:\Users\mirlo\AppData\Local\Temp\ipykernel_15964\3754038683.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  flights = pd.read_sql(query, con=connection)


,month,day_of_week,dep_del15,dep_time_blk,distance_group,segment_number,concurrent_flights,number_of_seats,carrier_name,airport_flights_month,...,previous_airport,prcp,snow,snwd,tmax,awnd,carrier_historical,dep_airport_hist,day_historical,dep_block_hist
0,9,4,0,1700-1759,4,5,64,143,Southwest Airlines Co.,21760,...,Chicago Midway International,0.0,0.0,0.0,760.0,1007.0,1.248290e+16,1.528070e+16,1.673210e+16,1.892080e+16
1,3,1,0,1600-1659,6,1,16,230,Frontier Airlines Inc.,5701,...,NONE,0.0,0.0,0.0,520.0,403.0,2.116660e+15,1.878670e+16,1.608680e+15,2.163590e+16
2,1,3,0,1400-1459,6,1,57,142,United Air Lines Inc.,14501,...,NONE,0.0,0.0,0.0,830.0,604.0,1.818820e+16,1.878670e+16,1.806070e+16,2.107150e+16
3,6,2,0,1600-1659,2,3,18,175,Southwest Airlines Co.,5661,...,Metropolitan Oakland International,0.0,0.0,0.0,830.0,492.0,2.597750e+15,2.377930e+16,2.323880e+16,3.277900e+16
4,8,6,0,1100-1159,1,3,19,100,JetBlue Airways,13118,...,Raleigh-Durham International,0.0,0.0,0.0,820.0,1297.0,3.166630e+15,2.284980e+16,1.618700e+16,1.651240e+16


#### Contexto Dataset 

Este Dataset fue extraido de la pagina web kaggle una plataforma de competencia de ciencia de datos y una comunidad en línea para científicos de datos y profesionales del aprendizaje automático de Google LLC.

Este Dataset contiene informacion sobre vuelos retrasados en el año 2019 con informacion sobre el clima, aereopuerto entre otros, para un mayor detalle puede visitar la pagina de donde salio en el siguiente enlace: https://www.kaggle.com/datasets/threnjen2019-airline-delays-and-cancellations?select=train.csv

##### Exploramos el Dataset para ver su contenido

Numero de filas y columnas 

In [7]:
num_rows, num_columns = flights.shape
print(f"Tamaño del Dataset: {num_rows} filas y {num_columns} columnas")

Tamaño del Dataset: 1048575 filas y 30 columnas


Revisamos el tipo de datos y la cantidad de valores unicos en el dataset 

In [14]:
summary = pd.DataFrame({
    'Tipo de Dato': flights.dtypes,
    'Valores Únicos': [flights[col].nunique() for col in flights.columns],
})

summary

,Tipo de Dato,Valores Únicos
month,int64,12
day_of_week,int64,7
dep_del15,int64,2
dep_time_blk,object,19
distance_group,int64,11
segment_number,int64,15
concurrent_flights,int64,107
number_of_seats,int64,82
carrier_name,object,17
airport_flights_month,int64,969


##### Descripcion de columnas 

1. Información Temporal:
- **MONTH:** Mes del año.
- **DAY_OF_WEEK:** Día de la semana.
- **DEP_BLOCK_HIST:** Bloque de salida.
2. Variables de Destino (Target):
- **DEP_DEL15:** Indicador binario de un retraso de salida de más de 15 minutos (1 es sí).
3. Características del Vuelo:
- **DISTANCE_GROUP:** Grupo de distancia que volará la aeronave.
- **SEGMENT_NUMBER:** Número de segmento en el que se encuentra la aeronave para el día.
- **CONCURRENT_FLIGHTS:** Número de vuelos simultáneos que salen del aeropuerto en el mismo bloque de salida.
- **NUMBER_OF_SEATS:** Número de asientos en la aeronave.
- **PLANE_AGE:** Antigüedad de la aeronave que sale.
4. Características del Transportista y Aeropuerto:
- **CARRIER_NAME:** Nombre del transportista.
- **AIRPORT_FLIGHTS_MONTH:** Promedio de vuelos del aeropuerto por mes.
- **AIRLINE_FLIGHTS_MONTH:** Promedio de vuelos de la aerolínea por mes.
- **AIRLINE_AIRPORT_FLIGHTS_MONTH:** Promedio de vuelos por mes para la aerolínea y el aeropuerto.
- **AVG_MONTHLY_PASS_AIRPORT:** Promedio de pasajeros para el aeropuerto de salida para el mes.
- **AVG_MONTHLY_PASS_AIRLINE:** Promedio de pasajeros por aerolínea para el mes.
- **FLT_ATTENDANTS_PER_PASS:** Auxiliares de vuelo por pasajero de la aerolínea.
- **GROUND_SERV_PER_PASS:** Empleados de servicio en tierra (mostrador de servicio) por pasajero de la aerolínea.
5. Información Geográfica:
- **DEPARTING_AIRPORT:** Aeropuerto de salida.
- **LATITUDE:** Latitud del aeropuerto de salida.
- **LONGITUDE:** Longitud del aeropuerto de salida.
- **PREVIOUS_AIRPORT:** Aeropuerto anterior del que salió la aeronave.
6. Condiciones Meteorológicas:
- **PRCP:** Pulgadas de precipitación del día.
- **SNOW:** Pulgadas de nevadas del día.
- **SNWD:** Pulgadas de nieve en tierra del día.
- **TMAX:** Temperatura máxima del día.
- **AWND:** Velocidad máxima del viento del día

#### Limpieza de Datos 

##### Verificamos la calidad de los datos

Miramos si hay valores nulos 

In [17]:
print("Valores Nulos:")
valores_nulos = pd.DataFrame(flights.isnull().sum())

valores_nulos

Valores Nulos:


,0
month,0
day_of_week,0
dep_del15,0
dep_time_blk,0
distance_group,0
segment_number,0
concurrent_flights,0
number_of_seats,0
carrier_name,0
airport_flights_month,0


Miramos si hay valores duplicados 

In [18]:
print("\nValores Duplicados:")
print(f"Número de filas duplicadas: {flights.duplicated().sum()}")


Valores Duplicados:
Número de filas duplicadas: 845


No se encontraron valores nulos, sin embargo se encontraron 845 datos duplicados por lo que vamos a proceder a eliminarlas para asegurar que asegurar que el  análisis no esté influenciado por registros redundantes

In [28]:
flights = flights.drop_duplicates()

##### Buscamos datos atipicos 

Para las columnas del dia de la semana y mes traemos el valor minimo y maximo de esas columnas para verificar que no hayan datos incoherentes 

In [29]:
max_month = flights['month'].max()
min_month = flights['month'].min()

max_day_of_week = flights['day_of_week'].max()
min_day_of_week = flights['day_of_week'].min()

max_dep_del15 = flights['dep_del15'].max()
min_dep_del15 = flights['dep_del15'].min()

print(f"Valor máximo de 'MONTH': {max_month}")
print(f"Valor mínimo de 'MONTH': {min_month}")
print(f"Valor máximo de 'DAY_OF_WEEK': {max_day_of_week}")
print(f"Valor mínimo de 'DAY_OF_WEEK': {min_day_of_week}")
print(f"Valor máximo de 'DEP_del15': {max_dep_del15}")
print(f"Valor mínimo de 'DEP_del15': {min_dep_del15}")

Valor máximo de 'MONTH': 12
Valor mínimo de 'MONTH': 1
Valor máximo de 'DAY_OF_WEEK': 7
Valor mínimo de 'DAY_OF_WEEK': 1
Valor máximo de 'DEP_del15': 1
Valor mínimo de 'DEP_del15': 0


Los datos son coherentes devido a que son 12 meses representados con numeros, y los 7 dias de la semana tambien representados con numeros 

Verificamos si  hay datos negativos para estas columnas: distance_group segment_number, concurrent_flights, number_of_seats, flt_attendants_per_pass,   ground_serv_per_pass

y en caso de que si, eliminarlos debido a que su precencia en estas seria ilogica 

In [30]:
columns_to_check = ['distance_group', 'segment_number', 'concurrent_flights', 'number_of_seats', 'flt_attendants_per_pass', 'ground_serv_per_pass', 'plane_age']

negative_values = flights[flights[columns_to_check].lt(0).any(axis=1)]

print("Valores negativos encontrados en las columnas especificadas:")
print(negative_values)

Valores negativos encontrados en las columnas especificadas:
Empty DataFrame
Columns: [month, day_of_week, dep_del15, dep_time_blk, distance_group, segment_number, concurrent_flights, number_of_seats, carrier_name, airport_flights_month, airline_flights_month, airline_airport_flights_month, avg_monthly_pass_airport, avg_monthly_pass_airline, flt_attendants_per_pass, ground_serv_per_pass, plane_age, departing_airport, latitude, longitude, previous_airport, prcp, snow, snwd, tmax, awnd, carrier_historical, dep_airport_hist, day_historical, dep_block_hist]
Index: []

[0 rows x 30 columns]


No encontramos valores negativos en estas columnas 

Hacemos lo mismo para las columnas: airport_flights_month, airline_flights_month, airline_airport_flights_month, avg_monthly_pass_airport, avg_monthly_pass_airline

In [31]:
columns_to_check = ['airport_flights_month', 'airline_flights_month', 'airline_airport_flights_month', 'avg_monthly_pass_airport', 'avg_monthly_pass_airline']

negative_values = flights[flights[columns_to_check].lt(0).any(axis=1)]

print("Valores negativos encontrados en las columnas especificadas:")
print(negative_values)

Valores negativos encontrados en las columnas especificadas:
Empty DataFrame
Columns: [month, day_of_week, dep_del15, dep_time_blk, distance_group, segment_number, concurrent_flights, number_of_seats, carrier_name, airport_flights_month, airline_flights_month, airline_airport_flights_month, avg_monthly_pass_airport, avg_monthly_pass_airline, flt_attendants_per_pass, ground_serv_per_pass, plane_age, departing_airport, latitude, longitude, previous_airport, prcp, snow, snwd, tmax, awnd, carrier_historical, dep_airport_hist, day_historical, dep_block_hist]
Index: []

[0 rows x 30 columns]


No encontramos valores negativos en estas columnas 

Mostramos los valores unicos y la cantidad de veces que salen de la columna carrier_name 

In [39]:
carrier_name_counts = flights['carrier_name'].value_counts().reset_index()
carrier_name_counts.columns = ['value', 'count']
print(carrier_name_counts)

                           value   count
0         Southwest Airlines Co.  208986
1           Delta Air Lines Inc.  152319
2         American Airlines Inc.  145919
3          United Air Lines Inc.   96959
4          SkyWest Airlines Inc.   94620
5          Midwest Airline. Inc.   48448
6                JetBlue Airways   43243
7           Alaska Airlines Inc.   38938
8   American Eagle Airlines Inc.   37053
9                    Comair Inc.   35322
10             Endeavor Air Inc.   32637
11              Spirit Air Lines   30101
12            Mesa Airlines Inc.   28690
13        Frontier Airlines Inc.   19715
14   Atlantic Southeast Airlines   15874
15        Hawaiian Airlines Inc.   12145
16                 Allegiant Air    6761


Tenemos que hay 16 aereolinias diferentes y nos muestra el número de vuelos asociados a cada aerolínea en el conjunto de datos.

Miramos la cantidad de valores unicos de las columnas departing_airport, y previus_airport

In [52]:
unique_values_departing_airport = flights['departing_airport'].nunique()

unique_values_previous_airport = flights['previous_airport'].nunique()

print(unique_values_departing_airport)
print(unique_values_previous_airport)

96
354


Este resultado nos dice que hay vuelos que salen de 95 aereopuertos diferentes (Devido a que un dato en esa columna no es un aereopuerto si no es NONE), y que hay 354 aereopuertos diferentes que figuran como el aereopuerto anterior en los datos 

La diferencia en la cantidad de valores únicos entre estas dos columnas refleja la diversidad de rutas y conexiones en los datos, con más variedad en los aeropuertos de origen previos en comparación con los aeropuertos de salida.

Revisamos la columna flt_attendants_per_pass

In [59]:
print(flights['flt_attendants_per_pass'].head(10))

0    6.178236e+10
1    1.157260e+16
2    2.538040e+16
3    6.178236e+10
4    1.600390e+15
5    6.178236e+10
6    1.441660e+15
7    1.441660e+15
8    3.484080e+16
9    6.178236e+10
Name: flt_attendants_per_pass, dtype: float64


Debido a que son datos demasiado grandes para representar auxiliares de vuelo por pasajero, la eliminamos del dataset 

In [60]:
flights = flights.drop(columns=['flt_attendants_per_pass'])

Revisamos la columna ground_serv_per_pass

In [61]:
print(flights['ground_serv_per_pass'].head(10))

0    9.889412e+10
1    7.134695e+09
2    2.289850e+16
3    9.889412e+10
4    1.268660e+16
5    9.889412e+10
6    1.486600e+15
7    1.486600e+15
8    1.068670e+16
9    9.889412e+10
Name: ground_serv_per_pass, dtype: float64


Debido a que son datos demasiado grandes para representar la cantidad de personal en tierra por pasajero, tambien la eliminamos del dataset 

In [62]:
flights = flights.drop(columns=['ground_serv_per_pass'])

Revisamos las columnas carrier_historical, dep_airport_hist, day_historical, dep_block_hist

In [64]:
columns_of_interest = ['carrier_historical', 'dep_airport_hist', 'day_historical', 'dep_block_hist']

print(flights[columns_of_interest].head())

   carrier_historical  dep_airport_hist  day_historical  dep_block_hist
0        1.248290e+16      1.528070e+16    1.673210e+16    1.892080e+16
1        2.116660e+15      1.878670e+16    1.608680e+15    2.163590e+16
2        1.818820e+16      1.878670e+16    1.806070e+16    2.107150e+16
3        2.597750e+15      2.377930e+16    2.323880e+16    3.277900e+16
4        3.166630e+15      2.284980e+16    1.618700e+16    1.651240e+16


Estos datos representan números extremadamente grandes y no parece representar datos como identificadores de aerolíneas, aeropuertos, días o bloques de salida. por lo tanto tambien eliminaremos esas columnas.

In [65]:
columns_to_remove = ['carrier_historical', 'dep_airport_hist', 'day_historical', 'dep_block_hist']

flights = flights.drop(columns=columns_to_remove)

Verificamos las columnas del dataset para corroborar la eliminacion de estas 

In [66]:
print(flights.columns)

Index(['month', 'day_of_week', 'dep_del15', 'dep_time_blk', 'distance_group',
       'segment_number', 'concurrent_flights', 'number_of_seats',
       'carrier_name', 'airport_flights_month', 'airline_flights_month',
       'airline_airport_flights_month', 'avg_monthly_pass_airport',
       'avg_monthly_pass_airline', 'plane_age', 'departing_airport',
       'latitude', 'longitude', 'previous_airport', 'prcp', 'snow', 'snwd',
       'tmax', 'awnd'],
      dtype='object')


Revisamos las variables de latitude y longitude 

In [67]:
columns_of_interest = ['longitude', 'latitude']

print(flights[columns_of_interest].head())

   longitude  latitude
0   -10488.0   39774.0
1   -90364.0   38747.0
2    -9534.0   29983.0
3   -90364.0   38747.0
4   -71006.0   42364.0


Los valores en las columnas latitude y longitude parecen estar escalados por un factor de 1000. En su formato actual, es probable que representen coordenadas geográficas, pero multiplicadas por 1000. Para convertirlas a su formato original o apropiado, dvamos a dividir cada valor por 1000.

In [68]:
flights['latitude'] = flights['latitude'] / 1000
flights['longitude'] = flights['longitude'] / 1000

Comprobamos el cambio

In [69]:
columns_of_interest = ['longitude', 'latitude']

print(flights[columns_of_interest].head())

   longitude  latitude
0    -10.488    39.774
1    -90.364    38.747
2     -9.534    29.983
3    -90.364    38.747
4    -71.006    42.364


Revisamos las columnas  prcp, snow, snwd, tmax, awnd



In [73]:
columns_of_interest = ['prcp', 'snow', 'snwd', 'tmax', 'awnd']

print(flights[columns_of_interest].head())

   prcp  snow  snwd   tmax    awnd
0   0.0   0.0   0.0  760.0  1007.0
1   0.0   0.0   0.0  520.0   403.0
2   0.0   0.0   0.0  830.0   604.0
3   0.0   0.0   0.0  830.0   492.0
4   0.0   0.0   0.0  820.0  1297.0


Los valores en las columnas PRCP (precipitación), SNOW (nieve), SNWD (profundidad de nieve), TMAX (temperatura máxima), y AWND (velocidad promedio del viento) parecen ser datos meteorológicos que pueden estar en un formato numérico específico.

Interpretación de los Valores:
prc (Precipitación en pulgadas): Los valores 0 y 1 indican la cantidad de precipitación. Si estos valores están en pulgadas, 1 podría representar 0.1 pulgadas, que es común en algunos registros meteorológicos.

snow (en décimas de pulgadas):  Un valor de 172 podría representar 17.2 pulgadas de nieve, lo cual es una cantidad considerable.

tmax (Temperatura máxima en décimas de grados Celsius): Los valores como 950 y 710 probablemente representan temperaturas en décimas de grados Celsius. Por ejemplo, 950 podría significar 95.0 °C, lo cual es extremadamente alto, lo que sugiere que estos valores tal vez necesiten un ajuste.

AWND (Velocidad promedio del viento en décimas de metros por segundo): Valores como 425 y 1141 podrían representar la velocidad del viento en décimas de metros por segundo. Por ejemplo, 425 significaría 42.5 m/s, que es una velocidad de viento extremadamente alta.

Vamos a dividir las columnas por 10 para la convercion de los valores a un formato mas comprencible y adecuado para hacer un analisis y una visualizacion de datos 

In [75]:
flights['prcp'] = flights['prcp'] / 10
flights['tmax'] = flights['tmax'] / 10
flights['awnd'] = flights['awnd'] / 10
flights['snow'] = flights['snow'] / 10

Revisamos las columnas para verificar la convercion 

In [76]:
columns_of_interest = ['prcp', 'snow', 'snwd', 'tmax', 'awnd']

print(flights[columns_of_interest].head())

   prcp  snow  snwd  tmax   awnd
0   0.0   0.0   0.0  76.0  100.7
1   0.0   0.0   0.0  52.0   40.3
2   0.0   0.0   0.0  83.0   60.4
3   0.0   0.0   0.0  83.0   49.2
4   0.0   0.0   0.0  82.0  129.7


#### Análisis Univariado:

#### Análisis Bivariado:

#### Análisis Multivariado

#### Resumen y Conclusiones:

#### Dataset Transformado 